# <center>Implementing LDA in Python</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [1]:
import numpy as np


#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [2]:
import os

def load_data(folder_path):
    texts = []
    # Walk through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):  # Only process .txt files
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read()
                texts.append(text)
    return texts


In [3]:
stopwords = stopwords.words("english")
print(stopwords)
stopwords.extend(['et', 'al','figure', 'paper', 'type', 'table'])
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
#data = load_data("data/ushmm_dn.json")["texts"]

#print (data[0][0:90])

# Use your folder path
folder_path = "data"
data = load_data(folder_path)

# Print first 90 characters of first text to verify
if data:
    print(data[0][0:90])

A Structural equation modeling analysis of generative AI chatbots adoption among students 


In [6]:
print(data[0][0:150])


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV", "PROPN"]):  # Added PROPN
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:150])


A Structural equation modeling analysis of generative AI chatbots adoption among students and educators in higher education

Abstract
In an era where 
structural equation model analysis generative AI chatbot adoption student educator high education Abstract era artificial intelligence AI reshape educ


In [7]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['structural', 'equation', 'model', 'analysis', 'generative', 'ai', 'chatbot', 'adoption', 'student', 'educator', 'high', 'education', 'abstract', 'era', 'artificial', 'intelligence', 'ai', 'reshape', 'educational', 'paradigm']


In [8]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]


data_words_nostops = remove_stopwords(data_words)

#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words_nostops, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words_nostops], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words_nostops)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])

['structural_equation', 'model', 'analysis', 'generative', 'ai', 'chatbot', 'adoption', 'student', 'educator', 'high', 'education', 'abstract', 'era', 'artificial_intelligence_ai', 'reshape', 'educational', 'paradigm', 'study', 'explore', 'ai']


In [9]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


In [10]:
# id2word = corpora.Dictionary(all_texts)

# corpus = []
# for text in data_words:
#     new = id2word.doc2bow(text)
#     corpus.append(new)

# print (corpus[0][0:20])

# word = id2word[[0][:1][0]]
# print (word)

In [10]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


In [11]:
test_doc = corpus[-1]

vector = lda_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.24791238), (1, 0.37759846), (2, 0.054194264), (3, 0.18293273), (4, 0.13736215)]
[(1, 0.37759846), (0, 0.24791238), (3, 0.18293273), (4, 0.13736215), (2, 0.054194264)]


In [12]:
lda_model.save("models/test_model.model")

In [13]:
new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [14]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.24791157), (1, 0.3775987), (2, 0.054194145), (3, 0.18293227), (4, 0.13736336)]
[(1, 0.3775987), (0, 0.24791157), (3, 0.18293227), (4, 0.13736336), (2, 0.054194145)]


## Vizualizing the Data

In [15]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.163892  0.194033       1        1  26.602860
4     -0.010985 -0.226395       2        1  23.472097
2     -0.233629 -0.076772       3        1  18.684119
3     -0.118492  0.196459       4        1  18.264870
0      0.199214 -0.087326       5        1  12.976054, topic_info=                   Term         Freq        Total Category  logprob  loglift
4514              genai  1386.000000  1386.000000  Default  30.0000  30.0000
150          assessment   604.000000   604.000000  Default  29.0000  29.0000
1669            teacher   787.000000   787.000000  Default  28.0000  28.0000
5560                gai   528.000000   528.000000  Default  27.0000  27.0000
477              design   275.000000   275.000000  Default  26.0000  26.0000
...                 ...          ...          ...      ...      ...      ...
1763         university    38.949722   308.854850   Topic5  -5.0881  -0.0285
1790           variable    29.101928   125.461298   Topic5  -5.3796   0.5809
2056  critical_thinking    26.870467   111.209105   Topic5  -5.4593   0.6217
1210        participant    33.675614   452.124630   Topic5  -5.2336  -0.5551
1669            teacher    31.225101   787.743448   Topic5  -5.3091  -1.1859

[314 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
6         3  0.641319    academic
6         5  0.355192    academic
10        3  0.204243  acceptance
10        4  0.379309  acceptance
10        5  0.418212  acceptance
...     ...       ...         ...
1833      1  0.767490       write
1833      5  0.226311       write
1834      3  0.737079     writing
1834      4  0.117106     writing
1834      5  0.144660     writing

[414 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 3, 4, 1])

In [16]:
print(data[64][0:90])

Challenges and Opportunities of Generative AI for Higher Education as Explained by ChatGPT
